# Python/FORTRAN interfacing teaser

This document is a collection of general ideas and starting points on how to simplify the further development of ocean, atmosphere and climate models. It might serve as discussion basis for the development and application of best practices in ocean/climate model development.

If you're experienced with both, Python and Fortran, you should be able to go on from here.

This notebook is meant to provide examples on how to access compiled FORTRAN routines using Python on-board packages/features. It covers
- Shared library compiling,
- shared library function calling
- and multi-dimensional array manipulation.

## A subjective ocean modeler's perspective

Ocean model code usually consists of a main program unit

```fortran
!--------------------------- main.f90 -----------------------------------
 PROGRAM main

  ! declaration

  ! initialization

  ! main loop
  do it = 1,itend
    CALL update_external_forcing
    CALL numerical_integration
    CALL write_disk_output
  end do

  ! restart preparation
    
 END PROGRAM main
!------------------------------------------------------------------------
```

and many subroutines organized in 

```fortran
!------------------------------ subroutines.f90 -------------------------

 ...

 SUBROUTINE square_root(a)
  IMPLICIT NONE
  real :: a
  a = SQRT(a)
 END SUBROUTINE square_root

 ...
 
!------------------------------------------------------------------------
```

## Testing during the development of ocean model code usually involves

* changing the FORTRAN code
* compiling of _complete_ model code
* setup of a test experiment directory structure
* job submission into an HPC queue
* ... waiting ...
* ... more waiting ...
* analysing model output
* drawing conclusions based on output
* probably starting over ...

It would be very useful if it could be shortened to

* changing FORTRAN code
* compiling only subroutine
* using only the subroutine with data manipulation and plotting attached
* drawing conclusions
* starting over ...

## Shared library compiling

ask your compiler's documentation about how to do it!

In [ ]:
%%writefile subroutines.f90

SUBROUTINE square(a)
    IMPLICIT NONE
    real :: a
    a = a**2
END SUBROUTINE square

SUBROUTINE square_root(b)
    IMPLICIT NONE
    real :: b
    b = SQRT(b)
END SUBROUTINE square_root


In [ ]:
%%script bash
echo $(which gfortran) && gfortran --version
gfortran -shared -fpic subroutines.f90 -o subroutines.so && ls -lrth

unix tools are useful...

In [ ]:
!readelf --symbols subroutines.so

## Python/FORTRAN interfacing

Don't worry, Numpy will handle most of this for you! However: https://docs.python.org/3/library/ctypes.html

In [ ]:
import ctypes as ctypes

fortran_routines = ctypes.CDLL('./subroutines.so')

In [ ]:
python_float = 4.0

In [ ]:
ctypes_float = ctypes.c_float(python_float)
ctypes_pointer = ctypes.pointer(ctypes_float)

_ = fortran_routines.square_(ctypes_pointer)
print(ctypes_pointer.contents, ':', python_float ** 2)

What happens here?

In [ ]:
_ = fortran_routines.square_root_(ctypes_pointer)
print(ctypes_pointer.contents, ':', python_float ** 0.5)

## Multi-dimensional array example

* https://docs.scipy.org/doc/numpy/reference/routines.ctypeslib.html
* https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.ctypes.html

In [1]:
%%writefile subroutines.f90

SUBROUTINE square(a,ij,ik)  !! global vars are bad!
    IMPLICIT NONE
    integer :: ij,ik
    real, dimension(ij,ik) :: a
    a = a**2
END SUBROUTINE square


Overwriting subroutines.f90


In [ ]:
%%script bash
gfortran -shared -fpic subroutines.f90 -o subroutines.so && ls -lrth
#readelf --symbols subroutines.so

In [ ]:
import numpy as np

fortran_routine_square = np.ctypeslib.load_library('subroutines.so', './').square_

In [ ]:
target_array = np.random.random((2, 2))
print(target_array)

In [ ]:
target_original = target_array.copy()

In [ ]:
# Ensure Fortran array and ensure single precision
target_array = np.asfortranarray(target_array, dtype=np.float32)
print(target_array.flags)

In [ ]:
ik = np.array(target_array.shape[0], dtype=np.int32)
ij = np.array(target_array.shape[1], dtype=np.int32)

In [ ]:
# Call by reference! So no meaningful output
fortran_routine_square(target_array.ctypes, ik.ctypes, ij.ctypes);

In [ ]:
print(target_array)

In [ ]:
print(target_original**2)

# Caveats: Beware of datatypes!
* https://docs.scipy.org/doc/numpy/user/basics.types.html
* e.g. https://northstar-www.dartmouth.edu/doc/solaris-forte/manuals/fortran/prog_guide/11_cfort.html

and use FORTRAN wrappers with ISO_C_BINDING declarations...

* get your datatypes clear!
* name mangling
* pass-by-value functionality

In [2]:
%%writefile wrappers.f90

SUBROUTINE wrapper_square(a,ij,ik) bind(c, name='wrapper_square')
    use iso_c_binding, only: c_int, c_float
    IMPLICIT NONE
    integer(c_int) :: ij,ik
    real(c_float), dimension(ij,ik) :: a
    call square(a,ij,ik)
END SUBROUTINE wrapper_square


Overwriting wrappers.f90


In [3]:
%%script bash
gfortran -shared -fpic wrappers.f90 subroutines.f90 -o subroutines.so && ls -lrth
readelf --symbols subroutines.so

total 36K
drwxr-xr-x 7 khoeflich khoeflich 4,0K Dez  5 11:30 RESOURCES
-rw-r--r-- 1 khoeflich khoeflich 1,8K Dez  5 11:53 wrapper.txt
-rw-r--r-- 1 khoeflich khoeflich  11K Dez  5 11:55 teaser.ipynb
-rw-r--r-- 1 khoeflich khoeflich  159 Dez  5 11:55 subroutines.f90
-rw-r--r-- 1 khoeflich khoeflich  253 Dez  5 11:55 wrappers.f90
-rwxr-xr-x 1 khoeflich khoeflich 7,5K Dez  5 11:55 subroutines.so

Symbol table '.dynsym' contains 12 entries:
   Num:    Value          Size Type    Bind   Vis      Ndx Name
     0: 0000000000000000     0 NOTYPE  LOCAL  DEFAULT  UND 
     1: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND __cxa_finalize
     2: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND _ITM_registerTMCloneTable
     3: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND _ITM_deregisterTMCloneTab
     4: 0000000000000000     0 NOTYPE  WEAK   DEFAULT  UND __gmon_start__
     5: 0000000000201028     0 NOTYPE  GLOBAL DEFAULT   19 _edata
     6: 0000000000201030     0 NOTYPE  GLOBAL DEFAUL

In [4]:
import numpy as np
fortran_routine_square = np.ctypeslib.load_library('subroutines.so','./').wrapper_square

In [5]:
target_array = np.random.random((2,2))
ik = np.array(target_array.shape[0],dtype=np.int32)
ij = np.array(target_array.shape[1],dtype=np.int32)

In [6]:
target_original = target_array.copy()
target_array = np.asfortranarray(target_array,dtype=np.float32)
fortran_routine_square(target_array.ctypes,ik.ctypes,ij.ctypes);

In [7]:
print(target_array)
print(target_original**2)

[[0.1843822  0.97012556]
 [0.08261189 0.784025  ]]
[[0.18438221 0.97012553]
 [0.08261189 0.78402494]]


# Questions

* possible to create thin interface to existing modules with ctypes only-use?